In [3]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import time
import itertools
import pickle

from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

path_main_folder = '/home/antorosi/Documents/Prediction'
sys.path.append(path_main_folder)

from conso.load_shape_data import load_data_conso, get_uniformed_data_conso, change_granularity, get_x_y_prediction_conso, get_train_test_sets, normalized_dataset, select_variables
from models.feedforward_NN import FeedForward

### Load and shape data 

In [4]:
# Load
path_data = os.path.join(path_main_folder, 'data')
dict_data_conso = load_data_conso(path_data)

# Uniformization
data_conso_df, dict_colnames_conso = get_uniformed_data_conso(dict_data_conso)

# Granularity from 15 min to 1H
data_conso_df = change_granularity(data_conso_df, granularity="1H")

# Get x and y from prediction
x_conso, y_conso, dict_colnames_conso = get_x_y_prediction_conso(data_conso_df, dict_colnames_conso, lags=[24])

### Cross-validation parameters 

In [5]:
# folder to store results
path_out = os.path.join(path_main_folder, 'out', 'cv_model_1.2')
if not os.path.exists(path_out):
    os.mkdir(path_out)

In [6]:
# variables used for input
#selected_variables = ['calendar', 'conso','meteo']
#gen_name = 'cmca'

selected_variables = ['conso','calendar','meteo']
gen_name = 'cmca'

In [7]:
# Test periods for each K step of the cross-validation
cv_periods = {}
cv_periods['period_1'] = (datetime.datetime(year=2013, month=1, day=1), datetime.datetime(year=2013, month=12, day=31))
cv_periods['period_2'] = (datetime.datetime(year=2014, month=1, day=1), datetime.datetime(year=2014, month=12, day=31))
cv_periods['period_3'] = (datetime.datetime(year=2015, month=1, day=1), datetime.datetime(year=2015, month=12, day=31))
cv_periods['period_4'] = (datetime.datetime(year=2016, month=1, day=1), datetime.datetime(year=2016, month=12, day=31))
cv_periods['period_5'] = (datetime.datetime(year=2017, month=1, day=1), datetime.datetime(year=2017, month=12, day=31))


In [8]:
# Getting each datasets
dict_datasets = {}
for key, date_period in cv_periods.items():
    x_conso_selected_var = select_variables(x_conso, dict_colnames_conso, selected_variables)
    dataset, dict_ds = get_train_test_sets(x_conso_selected_var, y_conso, date_period[0], date_period[1])
    dataset = normalized_dataset(dataset, dict_colnames_conso)
    
    dict_datasets[key] = {'dataset': dataset, 'dict_ds': dict_ds}

In [9]:
# model's parameters
nb_hidden_units = 151
nb_hidden_layers = 2
dropout = 0.1
input_dim = dict_datasets['period_1']['dataset']['train']['x'].shape[1]

training_epochs=400
batch_size=100

In [22]:
# Prepare results wrap up 
results_df = pd.DataFrame(columns=['name', 'layer_dims','dropout_rates','batchsize',
                                           'best_iter', 'train_mse',
                                           'train_mae', 'train_mape',
                                           'test_mse', 'test_mae',
                                           'test_mape'])
path_results = path_out

### Training

In [23]:
idx = 0

for name_period, el in dict_datasets.items():
    dataset = el['dataset']

    print('========================= Model {} ========================='.format(idx+1))

    # Prepare model characteristics
    name_model = '{}_FFNN_l{}*{}_d{}*{}_{}_norm'.format(name_period,nb_hidden_units, nb_hidden_layers,
                                                        dropout, nb_hidden_layers, 
                                                        gen_name)

    # Compile model
    model = FeedForward(name=name_model, output=path_out, input_dim=input_dim, output_dim=1, 
               l_dims=[nb_hidden_units]*nb_hidden_layers, dropout_rates=[dropout]*nb_hidden_layers,
               loss = 'mse', metrics = ['mape', 'mae'])

    # Prepare callbacks
    callbacks = []
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=250,
                                                       verbose=0, mode='auto')

    model_checkpoint = ModelCheckpoint(os.path.join(path_out, name_model, 'models', 'model-best.hdf5'),
                                               monitor='val_loss',
                                               verbose=0, save_best_only=True, save_weights_only=False,
                                               mode='auto', period=1)

    tensorboard_model = TensorBoard(log_dir=os.path.join(path_out, name_model, 'results', 'logs', time.strftime('%Y-%m-%d_%H:%M', time.localtime(time.time()))))
    tensorboard_summary = TensorBoard(log_dir=os.path.join(path_out, 'logs', name_model))


    callbacks.append(early_stop)
    callbacks.append(model_checkpoint)
    callbacks.append(tensorboard_model)
    callbacks.append(tensorboard_summary)

    # Train model
    model.main_train(dataset, training_epochs=training_epochs, batch_size=batch_size, callbacks=callbacks)

    # Get result and put it in results
    _, result = model.analyze_history(dataset)

    results_df= results_df.append(result, ignore_index=True)
    results_df.to_csv(os.path.join(path_results, 'cv_results.csv'), sep=';')

    # Reset graph
    K.clear_session()
    import tensorflow as tf
    tf.reset_default_graph()
    
    idx += 1


========================= Model 1 =========================
model: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_input (InputLayer)     (None, 151)               0         
_________________________________________________________________
dense_hidden_0 (Dense)       (None, 151)               22952     
_________________________________________________________________
dropout_hidden_0 (Dropout)   (None, 151)               0         
_________________________________________________________________
dense_hidden_1 (Dense)       (None, 151)               22952     
_________________________________________________________________
dropout_hidden_1 (Dropout)   (None, 151)               0         
_________________________________________________________________
dense_output (Dense)         (None, 1)                 152       
Total params: 46,056
Trainable params: 46,056
Non-trainable params: 0
____

Epoch 26/400
35112/35112 [==============================] - 1s 27us/step - loss: 15136981.6967 - mean_absolute_percentage_error: 5.7317 - mean_absolute_error: 3038.2937 - val_loss: 8677378.8014 - val_mean_absolute_percentage_error: 4.3063 - val_mean_absolute_error: 2215.7671
Epoch 27/400
35112/35112 [==============================] - 1s 28us/step - loss: 15087448.7754 - mean_absolute_percentage_error: 5.7185 - mean_absolute_error: 3030.1279 - val_loss: 8608106.8836 - val_mean_absolute_percentage_error: 4.3405 - val_mean_absolute_error: 2222.9954
Epoch 28/400
35112/35112 [==============================] - 1s 28us/step - loss: 15273822.4146 - mean_absolute_percentage_error: 5.7499 - mean_absolute_error: 3045.0049 - val_loss: 8515242.5719 - val_mean_absolute_percentage_error: 4.2722 - val_mean_absolute_error: 2209.0872
Epoch 29/400
35112/35112 [==============================] - 1s 27us/step - loss: 15214177.7419 - mean_absolute_percentage_error: 5.7411 - mean_absolute_error: 3044.4328 - v

Epoch 56/400
35112/35112 [==============================] - 1s 27us/step - loss: 14067787.0594 - mean_absolute_percentage_error: 5.4445 - mean_absolute_error: 2906.8092 - val_loss: 7650432.9386 - val_mean_absolute_percentage_error: 3.9204 - val_mean_absolute_error: 2023.6609
Epoch 57/400
35112/35112 [==============================] - 1s 26us/step - loss: 14131741.5425 - mean_absolute_percentage_error: 5.4325 - mean_absolute_error: 2904.8198 - val_loss: 7671403.6982 - val_mean_absolute_percentage_error: 3.9202 - val_mean_absolute_error: 2068.6908
Epoch 58/400
35112/35112 [==============================] - 1s 27us/step - loss: 13903513.2640 - mean_absolute_percentage_error: 5.3772 - mean_absolute_error: 2873.4488 - val_loss: 7565075.5402 - val_mean_absolute_percentage_error: 3.8772 - val_mean_absolute_error: 2047.5609
Epoch 59/400
35112/35112 [==============================] - 1s 27us/step - loss: 13749400.4674 - mean_absolute_percentage_error: 5.3399 - mean_absolute_error: 2855.9762 - v

Epoch 86/400
35112/35112 [==============================] - 1s 27us/step - loss: 12225701.7311 - mean_absolute_percentage_error: 4.9559 - mean_absolute_error: 2674.8181 - val_loss: 5837136.4716 - val_mean_absolute_percentage_error: 3.2234 - val_mean_absolute_error: 1692.1777
Epoch 87/400
35112/35112 [==============================] - 1s 27us/step - loss: 12185453.7270 - mean_absolute_percentage_error: 4.9285 - mean_absolute_error: 2657.7711 - val_loss: 5810142.6174 - val_mean_absolute_percentage_error: 3.1762 - val_mean_absolute_error: 1671.1294
Epoch 88/400
35112/35112 [==============================] - 1s 27us/step - loss: 12381063.3694 - mean_absolute_percentage_error: 4.9816 - mean_absolute_error: 2683.7584 - val_loss: 5866068.7895 - val_mean_absolute_percentage_error: 3.1945 - val_mean_absolute_error: 1685.4417
Epoch 89/400
35112/35112 [==============================] - 1s 27us/step - loss: 12206518.1482 - mean_absolute_percentage_error: 4.9242 - mean_absolute_error: 2657.6978 - v

Epoch 116/400
35112/35112 [==============================] - 1s 27us/step - loss: 12149555.6244 - mean_absolute_percentage_error: 4.9191 - mean_absolute_error: 2660.7389 - val_loss: 5684977.6463 - val_mean_absolute_percentage_error: 3.1337 - val_mean_absolute_error: 1665.4305
Epoch 117/400
35112/35112 [==============================] - 1s 27us/step - loss: 11912469.9370 - mean_absolute_percentage_error: 4.8545 - mean_absolute_error: 2625.6017 - val_loss: 5611322.3339 - val_mean_absolute_percentage_error: 3.1097 - val_mean_absolute_error: 1643.0155
Epoch 118/400
35112/35112 [==============================] - 1s 27us/step - loss: 11900624.9720 - mean_absolute_percentage_error: 4.8638 - mean_absolute_error: 2625.9693 - val_loss: 5716978.9222 - val_mean_absolute_percentage_error: 3.1346 - val_mean_absolute_error: 1653.0158
Epoch 119/400
35112/35112 [==============================] - 1s 26us/step - loss: 11955580.8876 - mean_absolute_percentage_error: 4.8825 - mean_absolute_error: 2636.3048

...

Epoch 122/400
35112/35112 [==============================] - 1s 27us/step - loss: 11740674.0335 - mean_absolute_percentage_error: 4.8057 - mean_absolute_error: 2611.4312 - val_loss: 6366578.8589 - val_mean_absolute_percentage_error: 3.3444 - val_mean_absolute_error: 1753.4900
Epoch 123/400
35112/35112 [==============================] - 1s 27us/step - loss: 11676686.1386 - mean_absolute_percentage_error: 4.8047 - mean_absolute_error: 2607.6729 - val_loss: 6240137.6482 - val_mean_absolute_percentage_error: 3.2252 - val_mean_absolute_error: 1686.5360
Epoch 124/400
35112/35112 [==============================] - 1s 27us/step - loss: 11674399.1070 - mean_absolute_percentage_error: 4.8103 - mean_absolute_error: 2612.1909 - val_loss: 6418884.6856 - val_mean_absolute_percentage_error: 3.3133 - val_mean_absolute_error: 1734.1415
Epoch 125/400
35112/35112 [==============================] - 1s 27us/step - loss: 11656020.7179 - mean_absolute_percentage_error: 4.8049 - mean_absolute_error: 2609.8591

Epoch 181/400
35112/35112 [==============================] - 1s 27us/step - loss: 11405429.6923 - mean_absolute_percentage_error: 4.7480 - mean_absolute_error: 2575.2854 - val_loss: 6318753.1610 - val_mean_absolute_percentage_error: 3.2641 - val_mean_absolute_error: 1709.9741
Epoch 182/400
35112/35112 [==============================] - 1s 27us/step - loss: 11467276.8249 - mean_absolute_percentage_error: 4.7760 - mean_absolute_error: 2594.3277 - val_loss: 6238003.9357 - val_mean_absolute_percentage_error: 3.2144 - val_mean_absolute_error: 1685.1725
Epoch 183/400
35112/35112 [==============================] - 1s 28us/step - loss: 11405015.5605 - mean_absolute_percentage_error: 4.7400 - mean_absolute_error: 2573.2849 - val_loss: 6328212.0473 - val_mean_absolute_percentage_error: 3.3027 - val_mean_absolute_error: 1736.5954
Epoch 184/400
35112/35112 [==============================] - 1s 27us/step - loss: 11566767.5042 - mean_absolute_percentage_error: 4.7663 - mean_absolute_error: 2593.3775

Epoch 240/400
35112/35112 [==============================] - 1s 27us/step - loss: 11309484.9617 - mean_absolute_percentage_error: 4.7220 - mean_absolute_error: 2562.8252 - val_loss: 6201484.3652 - val_mean_absolute_percentage_error: 3.2219 - val_mean_absolute_error: 1683.2306
Epoch 241/400
35112/35112 [==============================] - 1s 27us/step - loss: 11394047.8132 - mean_absolute_percentage_error: 4.7413 - mean_absolute_error: 2576.9539 - val_loss: 6422110.0312 - val_mean_absolute_percentage_error: 3.3187 - val_mean_absolute_error: 1739.1058
Epoch 242/400
35112/35112 [==============================] - 1s 27us/step - loss: 11371650.3434 - mean_absolute_percentage_error: 4.7414 - mean_absolute_error: 2578.9372 - val_loss: 6307628.3619 - val_mean_absolute_percentage_error: 3.2265 - val_mean_absolute_error: 1684.9080
Epoch 243/400
35112/35112 [==============================] - 1s 27us/step - loss: 11232370.7568 - mean_absolute_percentage_error: 4.7126 - mean_absolute_error: 2559.7707

Epoch 299/400
35112/35112 [==============================] - 1s 28us/step - loss: 11020578.1475 - mean_absolute_percentage_error: 4.6658 - mean_absolute_error: 2535.7250 - val_loss: 6210439.6396 - val_mean_absolute_percentage_error: 3.2300 - val_mean_absolute_error: 1695.8646
Epoch 300/400
35112/35112 [==============================] - 1s 27us/step - loss: 11154729.0995 - mean_absolute_percentage_error: 4.7002 - mean_absolute_error: 2556.6888 - val_loss: 6379666.7808 - val_mean_absolute_percentage_error: 3.2689 - val_mean_absolute_error: 1713.4676
Epoch 301/400
35112/35112 [==============================] - 1s 27us/step - loss: 11016866.6732 - mean_absolute_percentage_error: 4.6686 - mean_absolute_error: 2537.6132 - val_loss: 6335070.6119 - val_mean_absolute_percentage_error: 3.2341 - val_mean_absolute_error: 1695.0810
Epoch 302/400
35112/35112 [==============================] - 1s 27us/step - loss: 11050468.0738 - mean_absolute_percentage_error: 4.6833 - mean_absolute_error: 2544.0930

Epoch 358/400
35112/35112 [==============================] - 1s 27us/step - loss: 10940730.1061 - mean_absolute_percentage_error: 4.6411 - mean_absolute_error: 2526.2565 - val_loss: 6127512.6569 - val_mean_absolute_percentage_error: 3.1820 - val_mean_absolute_error: 1668.2705
Epoch 359/400
35112/35112 [==============================] - 1s 27us/step - loss: 10802001.1026 - mean_absolute_percentage_error: 4.6134 - mean_absolute_error: 2509.5156 - val_loss: 6108509.3533 - val_mean_absolute_percentage_error: 3.1786 - val_mean_absolute_error: 1662.6313
Epoch 360/400
35112/35112 [==============================] - 1s 27us/step - loss: 10837935.4882 - mean_absolute_percentage_error: 4.6376 - mean_absolute_error: 2520.9717 - val_loss: 6160658.0829 - val_mean_absolute_percentage_error: 3.1886 - val_mean_absolute_error: 1665.6596
Epoch 361/400
35112/35112 [==============================] - 1s 27us/step - loss: 10943100.6434 - mean_absolute_percentage_error: 4.6549 - mean_absolute_error: 2531.2862

In [24]:
with open(os.path.join(path_out, 'dict_datasets.pickle'),'wb') as f:
    pickle.dump(dict_datasets, f)